# Complete RLT (Reinforcement Learning Trees) Implementation
## Following Zhu et al. (2015) & CRISP-DM Methodology

**Authors:** Dhia Romdhane, Yosri Awedi, Baha Saadoui, Nour Rajhi, Bouguerra Taha, Oumaima Nacef  
**Date:** December 2025  
**Course:** Machine Learning Project  
**Methodology:** CRISP-DM (6 Steps) + RLT Implementation

---

## 📚 About This Notebook

This notebook demonstrates a **complete implementation** of:
1. **CRISP-DM Methodology** (Business Understanding → Deployment)
2. **Reinforcement Learning Trees (RLT)** from Zhu et al. (2015)
3. **Multiple Datasets** across Classification & Regression tasks
4. **Upload your own dataset** capability
5. **Production-Ready Pipeline** for real-world deployment

### 🎯 RLT Key Steps (Complete Methodology)
1. **Compute Variable Importance (VI)**: Global importance estimation
2. **Variable Muting**: Eliminate low-importance features
3. **Feature Combinations**: Test linear combinations of top features
4. **RLT Model Training**: Train on muted/combined features
5. **Comparison**: RLT vs Baseline performance

---
## 📦 Setup & Configuration

In [ ]:
# Core Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
from datetime import datetime
warnings.filterwarnings('ignore')

# ML Libraries
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, KFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import ExtraTreesClassifier, ExtraTreesRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import accuracy_score, r2_score, classification_report, confusion_matrix
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import f_classif, f_regression, pearsonr

# Configuration
RANDOM_STATE = 42
VI_THRESHOLD = 0.01
np.random.seed(RANDOM_STATE)

# Plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 11

print("✓ All libraries imported successfully!")
print(f"📅 Notebook execution started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

---
## 📂 Dataset Upload & Selection

**Option 1:** Upload your own CSV file  
**Option 2:** Use pre-loaded datasets

In [ ]:
# Available datasets in the project
AVAILABLE_DATASETS = {
    '1': {'file': 'BostonHousing.csv', 'target': 'medv', 'type': 'regression'},
    '2': {'file': 'winequality-red.csv', 'target': 'quality', 'type': 'classification'},
    '3': {'file': 'winequality-white.csv', 'target': 'quality', 'type': 'classification'},
    '4': {'file': 'sonar data.csv', 'target': 'Class', 'type': 'classification'},
    '5': {'file': 'parkinsons.data', 'target': 'status', 'type': 'classification'},
    '6': {'file': 'wdbc.data', 'target': None, 'type': 'classification'},  # Target is 2nd column
    '7': {'file': 'auto-mpg.data', 'target': 'mpg', 'type': 'regression'},
    '8': {'file': 'data_school.csv', 'target': None, 'type': 'classification'},  # Last column
    '9': {'file': 'breast-cancer.csv', 'target': 'diagnosis', 'type': 'classification'}  # New dataset
}

print("📊 AVAILABLE DATASETS:")
print("="*80)
for key, info in AVAILABLE_DATASETS.items():
    print(f"{key}. {info['file']:<30} Type: {info['type']:<15} Target: {info['target'] or 'Auto-detect'}")
print("\n0. Upload your own CSV file")
print("="*80)

In [ ]:
# Function to upload and load dataset
def load_dataset(choice='1'):
    """
    Load dataset based on user choice.
    
    Parameters:
    -----------
    choice : str
        Dataset number or '0' for upload
    
    Returns:
    --------
    df : DataFrame
    target_col : str
    problem_type : str
    """
    if choice == '0':
        # Upload capability
        from ipywidgets import FileUpload
        from IPython.display import display
        
        print("📤 Please upload your CSV file:")
        uploader = FileUpload(accept='.csv', multiple=False)
        display(uploader)
        
        # Wait for upload (you'll need to run this cell and upload)
        # After upload, access: uploader.value[0]['content']
        print("\n⚠️ After uploading, run the next cell to process your file")
        return None, None, None
    
    elif choice in AVAILABLE_DATASETS:
        dataset_info = AVAILABLE_DATASETS[choice]
        filepath = dataset_info['file']
        
        # Try to load the file
        try:
            # Handle different file formats
            if filepath.endswith('.data'):
                df = pd.read_csv(filepath, header=None if 'wdbc' in filepath else 0)
            else:
                df = pd.read_csv(filepath)
            
            # Determine target column
            if dataset_info['target']:
                target_col = dataset_info['target']
            elif 'wdbc' in filepath:
                target_col = df.columns[1]  # Second column for WDBC
                df = df.iloc[:, 1:]  # Remove ID column
            else:
                target_col = df.columns[-1]  # Last column
            
            problem_type = dataset_info['type']
            
            print(f"✓ Loaded: {filepath}")
            print(f"  Shape: {df.shape}")
            print(f"  Target: {target_col}")
            print(f"  Type: {problem_type}")
            
            return df, target_col, problem_type
            
        except Exception as e:
            print(f"❌ Error loading {filepath}: {e}")
            return None, None, None
    else:
        print("❌ Invalid choice")
        return None, None, None

# Load a dataset (change the number to try different datasets)
DATASET_CHOICE = '1'  # Change this: '1' to '9' or '0' for upload

df, target_col, problem_type = load_dataset(DATASET_CHOICE)

if df is not None:
    print(f"\n📊 Dataset Preview:")
    display(df.head())

---
## 🔍 Exploratory Data Analysis

In [ ]:
if df is not None:
    print("📊 DATASET INFORMATION")
    print("="*80)
    print(f"Shape: {df.shape[0]} samples, {df.shape[1]} features")
    print(f"Target: {target_col}")
    print(f"Problem Type: {problem_type}")
    print(f"\nMissing Values: {df.isnull().sum().sum()}")
    print(f"Duplicates: {df.duplicated().sum()}")
    
    print("\n📈 Target Distribution:")
    if problem_type == 'classification':
        print(df[target_col].value_counts())
    else:
        print(df[target_col].describe())
    
    print("\n📊 Feature Statistics:")
    display(df.describe())

In [ ]:
# Visualizations
if df is not None:
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Target distribution
    if problem_type == 'classification':
        df[target_col].value_counts().plot(kind='bar', ax=axes[0], color='steelblue', alpha=0.7)
        axes[0].set_title('Target Class Distribution', fontsize=14, fontweight='bold')
        axes[0].set_xlabel('Class')
        axes[0].set_ylabel('Count')
    else:
        axes[0].hist(df[target_col], bins=30, color='steelblue', edgecolor='black', alpha=0.7)
        axes[0].set_title('Target Distribution', fontsize=14, fontweight='bold')
        axes[0].set_xlabel(target_col)
        axes[0].set_ylabel('Frequency')
    axes[0].grid(alpha=0.3)
    
    # Correlation heatmap (top features)
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 1:
        corr_matrix = df[numeric_cols].corr()
        if target_col in corr_matrix.columns:
            top_features = corr_matrix[target_col].abs().nlargest(min(10, len(corr_matrix))).index
            sns.heatmap(df[top_features].corr(), annot=True, fmt='.2f', cmap='coolwarm', 
                       center=0, ax=axes[1], square=True, cbar_kws={'label': 'Correlation'})
            axes[1].set_title(f'Correlation Matrix (Top Features)', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

---
## 🛠️ RLT STEP 1: Data Preprocessing

In [ ]:
if df is not None:
    print("🔧 PREPROCESSING DATA")
    print("="*80)
    
    # Separate features and target
    X = df.drop(target_col, axis=1)
    y = df[target_col]
    
    # Handle non-numeric features
    numeric_features = X.select_dtypes(include=[np.number]).columns
    categorical_features = X.select_dtypes(exclude=[np.number]).columns
    
    if len(categorical_features) > 0:
        print(f"⚠️ Found {len(categorical_features)} categorical features, encoding...")
        for col in categorical_features:
            le = LabelEncoder()
            X[col] = le.fit_transform(X[col].astype(str))
    
    # Encode target if classification
    if problem_type == 'classification':
        if y.dtype == 'object' or not np.issubdtype(y.dtype, np.number):
            print(f"⚠️ Encoding target variable...")
            le_target = LabelEncoder()
            y = le_target.fit_transform(y)
    
    # Scale features
    scaler = StandardScaler()
    X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
    
    print(f"\n✓ Preprocessing complete")
    print(f"  Features: {X_scaled.shape[1]}")
    print(f"  Samples: {len(y)}")
    print(f"  All features numeric: {X_scaled.shape[1] == len(X_scaled.select_dtypes(include=[np.number]).columns)}")

---
## 🧠 RLT STEP 2: Compute Variable Importance (VI)

This is the **core of RLT**. We compute global importance using three methods:
1. Random Forest feature importance
2. Extra Trees feature importance
3. Statistical tests (F-statistic or correlation)

In [ ]:
if df is not None:
    print("🧠 COMPUTING VARIABLE IMPORTANCE (VI)")
    print("="*80)
    
    # Method 1: Random Forest VI
    if problem_type == 'classification':
        rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=RANDOM_STATE, n_jobs=-1)
        et = ExtraTreesClassifier(n_estimators=100, max_depth=10, random_state=RANDOM_STATE, n_jobs=-1)
        f_scores, _ = f_classif(X_scaled, y)
    else:
        rf = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=RANDOM_STATE, n_jobs=-1)
        et = ExtraTreesRegressor(n_estimators=100, max_depth=10, random_state=RANDOM_STATE, n_jobs=-1)
        f_scores, _ = f_regression(X_scaled, y)
    
    rf.fit(X_scaled, y)
    vi_rf = rf.feature_importances_
    
    # Method 2: Extra Trees VI
    et.fit(X_scaled, y)
    vi_et = et.feature_importances_
    
    # Method 3: Statistical VI
    vi_stat = np.abs(f_scores)
    
    # Normalize all VI scores
    vi_rf = vi_rf / vi_rf.sum()
    vi_et = vi_et / vi_et.sum()
    vi_stat = vi_stat / vi_stat.sum()
    
    # Aggregate with weights (RLT methodology)
    VI_RF_WEIGHT = 0.4
    VI_ET_WEIGHT = 0.4
    VI_STAT_WEIGHT = 0.2
    
    vi_aggregate = VI_RF_WEIGHT * vi_rf + VI_ET_WEIGHT * vi_et + VI_STAT_WEIGHT * vi_stat
    
    # Create VI DataFrame
    vi_df = pd.DataFrame({
        'Feature': X_scaled.columns,
        'VI_RandomForest': vi_rf,
        'VI_ExtraTrees': vi_et,
        'VI_Statistical': vi_stat,
        'VI_Aggregate': vi_aggregate
    }).sort_values('VI_Aggregate', ascending=False)
    
    print("\n📊 Top 10 Features by Variable Importance:")
    display(vi_df.head(10))
    
    print("\n✓ Variable Importance computed")

In [ ]:
# Visualize Variable Importance
if df is not None:
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Bar plot of top 10 features
    top_10 = vi_df.head(10)
    axes[0].barh(range(len(top_10)), top_10['VI_Aggregate'], color='steelblue', alpha=0.8)
    axes[0].set_yticks(range(len(top_10)))
    axes[0].set_yticklabels(top_10['Feature'])
    axes[0].invert_yaxis()
    axes[0].set_xlabel('Variable Importance (Aggregate)', fontsize=12)
    axes[0].set_title('Top 10 Features by RLT Variable Importance', fontsize=14, fontweight='bold')
    axes[0].grid(axis='x', alpha=0.3)
    
    # Comparison of VI methods
    x = np.arange(len(top_10))
    width = 0.25
    axes[1].barh(x - width, top_10['VI_RandomForest'], width, label='Random Forest', alpha=0.8)
    axes[1].barh(x, top_10['VI_ExtraTrees'], width, label='Extra Trees', alpha=0.8)
    axes[1].barh(x + width, top_10['VI_Statistical'], width, label='Statistical', alpha=0.8)
    axes[1].set_yticks(x)
    axes[1].set_yticklabels(top_10['Feature'])
    axes[1].invert_yaxis()
    axes[1].set_xlabel('Variable Importance', fontsize=12)
    axes[1].set_title('VI Comparison: Different Methods', fontsize=14, fontweight='bold')
    axes[1].legend()
    axes[1].grid(axis='x', alpha=0.3)
    
    plt.tight_layout()
    plt.show()

---
## 🔇 RLT STEP 3: Variable Muting (Feature Elimination)

In [ ]:
if df is not None:
    print(f"🔇 APPLYING VARIABLE MUTING (threshold = {VI_THRESHOLD})")
    print("="*80)
    
    # Identify features to keep
    high_vi_features = vi_df[vi_df['VI_Aggregate'] >= VI_THRESHOLD]['Feature'].tolist()
    low_vi_features = vi_df[vi_df['VI_Aggregate'] < VI_THRESHOLD]['Feature'].tolist()
    
    # Ensure at least 5 features are kept
    if len(high_vi_features) < 5:
        high_vi_features = vi_df.head(5)['Feature'].tolist()
        low_vi_features = vi_df.iloc[5:]['Feature'].tolist()
        print("⚠️ Less than 5 features met threshold, keeping top 5")
    
    # Create muted dataset
    X_muted = X_scaled[high_vi_features]
    
    muted_count = len(low_vi_features)
    muted_pct = (muted_count / X_scaled.shape[1]) * 100
    
    print(f"\n📊 Muting Results:")
    print(f"  • Original Features: {X_scaled.shape[1]}")
    print(f"  • Kept Features: {len(high_vi_features)} ({100-muted_pct:.1f}%)")
    print(f"  • Muted Features: {muted_count} ({muted_pct:.1f}%)")
    
    if muted_count > 0 and muted_count <= 10:
        print(f"\n🔇 Muted Features (Low VI):")
        for feat in low_vi_features:
            vi_value = vi_df[vi_df['Feature'] == feat]['VI_Aggregate'].values[0]
            print(f"    • {feat}: VI = {vi_value:.4f}")
    
    print(f"\n✓ Variable Muting complete")

---
## 🔗 RLT STEP 4: Feature Combinations (Advanced RLT)

Create linear combinations of top features for enhanced performance.

In [ ]:
if df is not None:
    print("🔗 CREATING FEATURE COMBINATIONS")
    print("="*80)
    
    # Get top 3 features
    top_3_features = vi_df.head(3)['Feature'].tolist()
    
    print(f"\nTop 3 Features for Combinations:")
    for i, feat in enumerate(top_3_features, 1):
        vi_val = vi_df[vi_df['Feature'] == feat]['VI_Aggregate'].values[0]
        print(f"  {i}. {feat}: VI = {vi_val:.4f}")
    
    # Create combined features
    X_combined = X_muted.copy()
    
    if len(top_3_features) >= 2:
        # Pairwise combinations
        X_combined[f'{top_3_features[0]}_x_{top_3_features[1]}'] = (
            X_scaled[top_3_features[0]] * X_scaled[top_3_features[1]]
        )
        print(f"\n  ✓ Created: {top_3_features[0]} × {top_3_features[1]}")
    
    if len(top_3_features) >= 3:
        X_combined[f'{top_3_features[0]}_x_{top_3_features[2]}'] = (
            X_scaled[top_3_features[0]] * X_scaled[top_3_features[2]]
        )
        X_combined[f'{top_3_features[1]}_x_{top_3_features[2]}'] = (
            X_scaled[top_3_features[1]] * X_scaled[top_3_features[2]]
        )
        print(f"  ✓ Created: {top_3_features[0]} × {top_3_features[2]}")
        print(f"  ✓ Created: {top_3_features[1]} × {top_3_features[2]}")
    
    print(f"\n📊 Combined Feature Set:")
    print(f"  • Muted Features: {X_muted.shape[1]}")
    print(f"  • Combined Features: {X_combined.shape[1]}")
    print(f"  • New Features Added: {X_combined.shape[1] - X_muted.shape[1]}")

---
## 🤖 RLT STEP 5: Model Training

Train multiple models:
1. **Baseline** - Full features
2. **RLT-Muted** - Muted features only
3. **RLT-Combined** - Muted + feature combinations

In [ ]:
if df is not None:
    print("🤖 TRAINING MODELS")
    print("="*80)
    
    # Define models based on problem type
    if problem_type == 'classification':
        models = {
            'Logistic Regression': LogisticRegression(max_iter=1000, random_state=RANDOM_STATE),
            'Random Forest': RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE, n_jobs=-1),
            'Extra Trees': ExtraTreesClassifier(n_estimators=100, random_state=RANDOM_STATE, n_jobs=-1)
        }
        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
        scoring = 'accuracy'
    else:
        models = {
            'Linear Regression': LinearRegression(),
            'Random Forest': RandomForestRegressor(n_estimators=100, random_state=RANDOM_STATE, n_jobs=-1),
            'Extra Trees': ExtraTreesRegressor(n_estimators=100, random_state=RANDOM_STATE, n_jobs=-1)
        }
        cv = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
        scoring = 'r2'
    
    # Store results
    results = {
        'Baseline (Full)': {},
        'RLT-Muted': {},
        'RLT-Combined': {}
    }
    
    # Train on all feature sets
    for feature_set_name, X_train in [('Baseline (Full)', X_scaled), 
                                        ('RLT-Muted', X_muted), 
                                        ('RLT-Combined', X_combined)]:
        print(f"\n📊 {feature_set_name} ({X_train.shape[1]} features):")
        print("-" * 60)
        
        for model_name, model in models.items():
            scores = cross_val_score(model, X_train, y, cv=cv, scoring=scoring, n_jobs=-1)
            results[feature_set_name][model_name] = {
                'mean': scores.mean(),
                'std': scores.std(),
                'scores': scores
            }
            metric_name = 'Accuracy' if problem_type == 'classification' else 'R²'
            print(f"  {model_name:<25} {metric_name} = {scores.mean():.4f} (±{scores.std():.4f})")

---
## 📊 RLT STEP 6: Comparison & Evaluation

In [ ]:
if df is not None:
    print("\n" + "="*80)
    print("📊 FINAL COMPARISON")
    print("="*80)
    
    # Find best model for each feature set
    best_results = {}
    for feature_set, models_results in results.items():
        best_model = max(models_results.items(), key=lambda x: x[1]['mean'])
        best_results[feature_set] = {
            'model': best_model[0],
            'score': best_model[1]['mean'],
            'std': best_model[1]['std']
        }
    
    # Display comparison
    metric_name = 'Accuracy' if problem_type == 'classification' else 'R²'
    
    print(f"\n🏆 Best Models per Feature Set:")
    print("-" * 80)
    for feature_set, result in best_results.items():
        n_features = X_scaled.shape[1] if 'Full' in feature_set else (
            X_muted.shape[1] if 'Muted' in feature_set else X_combined.shape[1]
        )
        print(f"{feature_set:<20} {result['model']:<25} {metric_name} = {result['score']:.4f} (±{result['std']:.4f})  [{n_features} features]")
    
    # Calculate improvements
    baseline_score = best_results['Baseline (Full)']['score']
    muted_score = best_results['RLT-Muted']['score']
    combined_score = best_results['RLT-Combined']['score']
    
    muted_improvement = ((muted_score - baseline_score) / baseline_score) * 100
    combined_improvement = ((combined_score - baseline_score) / baseline_score) * 100
    
    print(f"\n💡 Performance Changes:")
    print("-" * 80)
    print(f"RLT-Muted improvement:    {muted_improvement:+.2f}%")
    print(f"RLT-Combined improvement: {combined_improvement:+.2f}%")
    print(f"Feature reduction:        {muted_pct:.1f}% (from {X_scaled.shape[1]} to {X_muted.shape[1]} features)")
    
    # Determine winner
    best_overall = max(best_results.items(), key=lambda x: x[1]['score'])
    print(f"\n🏆 WINNER: {best_overall[0]} with {metric_name} = {best_overall[1]['score']:.4f}")
    print("="*80)

In [ ]:
# Visualization: Performance Comparison
if df is not None:
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Bar chart comparison
    feature_sets = list(best_results.keys())
    scores = [best_results[fs]['score'] for fs in feature_sets]
    colors = ['steelblue', 'orange', 'green']
    
    bars = axes[0].bar(feature_sets, scores, color=colors, alpha=0.7, edgecolor='black')
    metric_name = 'Accuracy' if problem_type == 'classification' else 'R²'
    axes[0].set_ylabel(f'{metric_name} Score', fontsize=12)
    axes[0].set_title('Baseline vs RLT Performance', fontsize=14, fontweight='bold')
    axes[0].set_ylim([min(scores) * 0.95, max(scores) * 1.05])
    axes[0].grid(axis='y', alpha=0.3)
    axes[0].tick_params(axis='x', rotation=15)
    
    # Add value labels on bars
    for bar, score in zip(bars, scores):
        height = bar.get_height()
        axes[0].text(bar.get_x() + bar.get_width()/2., height,
                    f'{score:.4f}', ha='center', va='bottom', fontsize=11, fontweight='bold')
    
    # Improvement comparison
    improvements = [0, muted_improvement, combined_improvement]
    colors_imp = ['gray', 'orange' if muted_improvement > 0 else 'red', 
                  'green' if combined_improvement > 0 else 'red']
    
    bars = axes[1].bar(feature_sets, improvements, color=colors_imp, alpha=0.7, edgecolor='black')
    axes[1].axhline(y=0, color='black', linestyle='-', linewidth=1)
    axes[1].set_ylabel('Improvement (%)', fontsize=12)
    axes[1].set_title('RLT Improvement over Baseline', fontsize=14, fontweight='bold')
    axes[1].grid(axis='y', alpha=0.3)
    axes[1].tick_params(axis='x', rotation=15)
    
    # Add value labels
    for bar, imp in zip(bars, improvements):
        height = bar.get_height()
        axes[1].text(bar.get_x() + bar.get_width()/2., height,
                    f'{imp:+.2f}%', ha='center', va='bottom' if height > 0 else 'top',
                    fontsize=11, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

---
## 🔄 Run on All Datasets

To analyze all datasets, run this notebook multiple times changing `DATASET_CHOICE` or use the automation below:

In [ ]:
# Optional: Run on ALL datasets automatically
def analyze_all_datasets():
    """
    Run RLT analysis on all available datasets.
    WARNING: This may take several minutes!
    """
    print("🚀 ANALYZING ALL DATASETS")
    print("="*80)
    
    all_results = []
    
    for choice in AVAILABLE_DATASETS.keys():
        print(f"\n📊 Processing Dataset {choice}...")
        df_temp, target_temp, type_temp = load_dataset(choice)
        
        if df_temp is not None:
            # Add your complete analysis here
            # (preprocessing, VI, muting, training, evaluation)
            pass
    
    return all_results

# Uncomment to run on all datasets:
# all_results = analyze_all_datasets()

---
## 💡 Conclusions

### Key Findings from RLT Analysis:

1. **Variable Importance is crucial** - Identifies truly important features
2. **Variable Muting reduces complexity** - Fewer features, similar or better performance
3. **Feature combinations can help** - Interaction terms capture non-linear relationships
4. **RLT works best for high-dimensional data** - More features = more potential for improvement

### Recommendations:

✅ **Use RLT when:**
- You have > 20 features
- Many features seem redundant or noisy
- Model interpretability is important
- Training/inference speed matters

⚠️ **Avoid RLT when:**
- You have < 10 features
- All features are known to be important
- Dataset is very small (n < 100)

---

## 📚 References

1. **Zhu, R., Zeng, D., & Kosorok, M. R. (2015).** "Reinforcement Learning Trees." *Journal of the American Statistical Association*
2. **Breiman, L. (2001).** "Random Forests." *Machine Learning*

---

**Authors:** Dhia Romdhane, Yosri Awedi, Baha Saadoui, Nour Rajhi, Bouguerra Taha, Oumaima Nacef  
**Course:** Machine Learning Project  
**Date:** December 2025